<a href="https://colab.research.google.com/github/Paradoxxs/Paradoxxs.github.io/blob/main/Simple_PE_scanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import pickle
import sklearn.ensemble as ske
from sklearn.model_selection import train_test_split
from sklearn import tree, linear_model
from sklearn.feature_selection import SelectFromModel
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix

Load data

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Data science/Data/antivirus_demo_data.csv', sep='|')
X = data.drop(['Name', 'md5', 'legitimate'], axis=1).values
y = data['legitimate'].values

print('Researching important feature based on %i total features\n' % X.shape[1])

Researching important feature based on 54 total features



Feature selection using Trees Classifier

In [ ]:
fsel = ske.ExtraTreesClassifier().fit(X, y)
model = SelectFromModel(fsel, prefit=True)
X_new = model.transform(X)
nb_features = X_new.shape[1]

Split the data into to test and train

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y ,test_size=0.2)

In [ ]:
features = []

print('%i features identified as important:' % nb_features)

13 features identified as important:


In [ ]:
indices = np.argsort(fsel.feature_importances_)[::-1][:nb_features]
for f in range(nb_features):
    print("%d. feature %s (%f)" % (f + 1, data.columns[2+indices[f]], fsel.feature_importances_[indices[f]]))




1. feature DllCharacteristics (0.152100)
2. feature Machine (0.104096)
3. feature SectionsMaxEntropy (0.073897)
4. feature VersionInformationSize (0.073672)
5. feature Characteristics (0.067896)
6. feature Subsystem (0.063009)
7. feature MajorSubsystemVersion (0.062743)
8. feature ImageBase (0.050253)
9. feature ResourcesMaxEntropy (0.044595)
10. feature ResourcesMinEntropy (0.043756)
11. feature SizeOfOptionalHeader (0.043518)
12. feature MajorOperatingSystemVersion (0.025248)
13. feature SectionsMinEntropy (0.020449)


Defines the algorithm for comparison inside a dictionary

In [ ]:

algorithms = {
        "DecisionTree": tree.DecisionTreeClassifier(max_depth=10),
        "RandomForest": ske.RandomForestClassifier(n_estimators=50),
        "GradientBoosting": ske.GradientBoostingClassifier(n_estimators=50),
        "AdaBoost": ske.AdaBoostClassifier(n_estimators=100),
    }

For loop for testing the different algorithms
using a list comprehension over dictionary

In [ ]:
results = {}
print("\nNow testing algorithms")
results = {k:v.fit(X_train, y_train).score(X_test, y_test) for (k,v) in algorithms.items()}
print(results)


Now testing algorithms
{'DecisionTree': 0.9912712785222746, 'RandomForest': 0.9943498732343354, 'GradientBoosting': 0.9883375588554871, 'AdaBoost': 0.9867077145961608}


Printing out the algroithm with the higest score.

In [ ]:
winner = max(results, key=results.get)
print(' %s with a %f %%' % (winner, results[winner]*100))

 RandomForest with a 99.434987 %


In [ ]:
# Identify false and true positive rates
clf = algorithms[winner]
res = clf.predict(X_test)
mt = confusion_matrix(y_test, res)
print(mt)
print("False positive rate %f %%" % ((mt[0][1] / float(sum(mt[0])))*100))
print('False negative rate %f %%' % ( (mt[1][0] / float(sum(mt[1]))*100)))

[[19207    87]
 [   69  8247]]
False positive rate 0.450917 %
False negative rate 0.829726 %
